# Problem Set 1

## Preparation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import pytz

###Import data

In [2]:
# You will need the shared Google drive https://drive.google.com/drive/u/1/folders/0ALmMMRrzz_-5Uk9PVA
from google.colab import drive
drive.mount("/content/drive")
!mkdir -p datashim
!cp /content/drive/Shareddrives/DS602-S22/Data/data602.py ./datashim
from datashim import data602

Mounted at /content/drive


In [3]:
cdf = data602.read_citations('/content/drive/Shareddrives/DS602-S22/Data/Parking_and_Moving_Citations.csv')
wdf = data602.read_weather('/content/drive/Shareddrives/DS602-S22/Data/weather_clean.csv')

## Question 1

Using the `weather` dataset, provide the following summary statistics:

In [4]:
wdf.head()

,date,wnd_speed,tmp_meas,precip,precip_intensity,cloudcover,tstorm,winter,mist,fog
0,2015-12-31 19:54:00-05:00,1.5,7.2,0.0,0,4.0,0,0,0,0
1,2015-12-31 20:54:00-05:00,3.1,6.1,0.0,0,4.0,0,0,0,0
2,2015-12-31 21:54:00-05:00,2.1,6.7,0.0,0,4.0,0,0,0,0
3,2015-12-31 22:54:00-05:00,2.6,6.1,0.0,0,4.0,0,0,0,0
4,2015-12-31 23:54:00-05:00,2.1,5.6,0.0,0,4.0,0,0,0,0


###a. Over the entire datasets, which observations had the hottest and coldest temperatures? What were those temperatures?

In [5]:
HT=wdf["tmp_meas"].max() ## to find the maximum temperature
CT=wdf["tmp_meas"].min() ## to find the minimum temperature
print("Hottest Temperature is:",HT)
print("Coldest Temperature is:",CT)


Hottest Temperature is: 37.8
Coldest Temperature is: -16.1


### b. In 2020, what month had the hottest average temperature? The coldest?

In [47]:
wdf['day'] = wdf['date'].dt.day ## to change the timestamp and created 3 different columns for day month year
wdf['month'] = wdf['date'].dt.month
wdf['year'] = wdf['date'].dt.year
wdf

,date,wnd_speed,tmp_meas,precip,precip_intensity,cloudcover,tstorm,winter,mist,fog,Dates,Time,day,month,year
0,2015-12-31 19:54:00-05:00,1.5,7.2,0.0,0,4.0,0,0,0,0,2015-12-31,19:54:00,31,12,2015
1,2015-12-31 20:54:00-05:00,3.1,6.1,0.0,0,4.0,0,0,0,0,2015-12-31,20:54:00,31,12,2015
2,2015-12-31 21:54:00-05:00,2.1,6.7,0.0,0,4.0,0,0,0,0,2015-12-31,21:54:00,31,12,2015
3,2015-12-31 22:54:00-05:00,2.6,6.1,0.0,0,4.0,0,0,0,0,2015-12-31,22:54:00,31,12,2015
4,2015-12-31 23:54:00-05:00,2.1,5.6,0.0,0,4.0,0,0,0,0,2015-12-31,23:54:00,31,12,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48554,2021-07-16 15:54:00-04:00,4.6,36.7,0.0,0,2.0,0,0,0,0,2021-07-16,15:54:00,16,7,2021
48555,2021-07-16 16:54:00-04:00,4.1,35.6,0.0,0,2.0,0,0,0,0,2021-07-16,16:54:00,16,7,2021
48556,2021-07-16 17:54:00-04:00,4.1,33.3,0.0,0,3.0,0,0,0,0,2021-07-16,17:54:00,16,7,2021
48557,2021-07-16 18:54:00-04:00,3.6,32.8,0.0,0,2.0,0,0,0,0,2021-07-16,18:54:00,16,7,2021


In [8]:
wdf_2020 = wdf[wdf["year"]==2020] #Seperating the year 2020 from the complete dataset
wdf_2020.tail()

,date,wnd_speed,tmp_meas,precip,precip_intensity,cloudcover,tstorm,winter,mist,fog,Dates,Time,day,month,year
43832,2020-12-31 19:54:00-05:00,2.6,5.6,0.0,0,4.0,0,0,0,0,2020-12-31,19:54:00,31,12,2020
43833,2020-12-31 20:54:00-05:00,2.6,5.0,0.0,0,4.0,0,0,0,0,2020-12-31,20:54:00,31,12,2020
43834,2020-12-31 21:54:00-05:00,2.1,3.9,0.0,0,2.0,0,0,0,0,2020-12-31,21:54:00,31,12,2020
43835,2020-12-31 22:54:00-05:00,2.1,3.3,0.0,0,2.0,0,0,0,0,2020-12-31,22:54:00,31,12,2020
43836,2020-12-31 23:54:00-05:00,1.5,2.8,0.0,0,1.0,0,0,0,0,2020-12-31,23:54:00,31,12,2020


In [9]:
mean_2020=wdf_2020.groupby(["year", "month"])['tmp_meas'].mean().reset_index() #Finding the mean for the temperatures
mean_2020.head()

,year,month,tmp_meas
0,2020,1,4.853495
1,2020,2,5.502730
2,2020,3,10.436205
3,2020,4,11.843472
4,2020,5,16.923822


In [10]:
hottest_month=mean_2020.loc[mean_2020['tmp_meas'] ==mean_2020["tmp_meas"].max()]['month'].values[0]
coldest_month=mean_2020.loc[mean_2020['tmp_meas'] ==mean_2020["tmp_meas"].min()]['month'].values[0]
print("Hottest Month in 2020 is :",hottest_month)
print("Coldest Month in 2020 is :",coldest_month)

Hottest Month in 2020 is : 7
Coldest Month in 2020 is : 12


### c. In 2020, how many days were associated with rain (i.e., liquid precipitation exceeding zero for at least one observation)?

In [11]:
rainy_days=wdf_2020.loc[(wdf_2020['precip'] > 0.0)] # if the precepitation is greater than 0 then we consider it as a raint day
op=rainy_days.groupby(["year", "month","day"])['precip'].sum().reset_index()
print('Number of Rainy Days in 2020:',len(op))

Number of Rainy Days in 2020: 144


##Question 2
From the `citations` dataset, provide the following summary statistics:

In [12]:
cdf.head()

,citation,tag,state,violcode,description,violfine,violdate,location,violdtl
0,91,03W671,MD,27,No Stop/Park Street Cleaning,52,2011-12-09 11:55:00-05:00,<NA>,2011-12-09 11:55:00
1,166,T294093T,MD,12,No Stopping/Standing Not Tow-Away Zone,32,2011-12-09 12:39:00-05:00,<NA>,2011-12-09 12:39:00
2,299,19TLR318,ME,99,All Other Stopping or Parking Violations,32,2011-12-09 11:55:00-05:00,<NA>,2011-12-09 11:55:00
3,315,14TLR551,ME,99,All Other Stopping or Parking Violations,32,2011-12-09 12:02:00-05:00,<NA>,2011-12-09 12:02:00
4,562,9AE2340,MD,18,All Other Parking Meter Violations,32,2011-12-12 10:47:00-05:00,LAFAYETTE MARKET LOT Baltimore MD,2011-12-12 10:47:00


###a. Which violation types, on average, incur the highest fines?

In [13]:
maximum_fines=cdf.groupby(["description"])['violfine'].mean().reset_index() #finding the mean for all the fines and associating it with to the description
res=maximum_fines.loc[maximum_fines['violfine']==max(maximum_fines['violfine'])]
print('Violations which has highest fine is\n',res)

Violations which has highest fine is
                                  description  violfine
30  Unlawful Dumping/Waste Hauler w/o Permit     502.0
39          No Parking/Standing Vendor Truck     502.0


###b. In 2020, which violation type was the most common? What percentage of total citations issued that year did it comprise?

In [14]:
cdf['day'] = cdf['violdtl'].dt.day ## to change the timestamp and created 3 different columns for day month year
cdf['month'] = cdf['violdtl'].dt.month
cdf['year'] = cdf['violdtl'].dt.year
cdf.head()

,citation,tag,state,violcode,description,violfine,violdate,location,violdtl,day,month,year
0,91,03W671,MD,27,No Stop/Park Street Cleaning,52,2011-12-09 11:55:00-05:00,<NA>,2011-12-09 11:55:00,9,12,2011
1,166,T294093T,MD,12,No Stopping/Standing Not Tow-Away Zone,32,2011-12-09 12:39:00-05:00,<NA>,2011-12-09 12:39:00,9,12,2011
2,299,19TLR318,ME,99,All Other Stopping or Parking Violations,32,2011-12-09 11:55:00-05:00,<NA>,2011-12-09 11:55:00,9,12,2011
3,315,14TLR551,ME,99,All Other Stopping or Parking Violations,32,2011-12-09 12:02:00-05:00,<NA>,2011-12-09 12:02:00,9,12,2011
4,562,9AE2340,MD,18,All Other Parking Meter Violations,32,2011-12-12 10:47:00-05:00,LAFAYETTE MARKET LOT Baltimore MD,2011-12-12 10:47:00,12,12,2011


In [15]:
cdf_2020 = cdf[cdf["year"]==2020] # seperating the year 2020 
cdf_2020.head()

,citation,tag,state,violcode,description,violfine,violdate,location,violdtl,day,month,year
4558,312801,1DZ9619,MD,99,All Other Stopping or Parking Violations,32,2020-09-30 11:55:00-04:00,<NA>,2020-09-30 11:55:00,30,9,2020
4559,312827,1EG9644,MD,99,All Other Stopping or Parking Violations,32,2020-09-30 13:30:00-04:00,"(39.29192061, -76.62030785)",2020-09-30 13:30:00,30,9,2020
4560,312843,7MD4370,MD,22,Expired Tags,32,2020-10-01 11:30:00-04:00,<NA>,2020-10-01 11:30:00,1,10,2020
4561,312850,4EF2475,MD,99,All Other Stopping or Parking Violations,32,2020-10-01 11:35:00-04:00,<NA>,2020-10-01 11:35:00,1,10,2020
4562,312876,4EA9361,MD,99,All Other Stopping or Parking Violations,32,2020-10-01 11:51:00-04:00,<NA>,2020-10-01 11:51:00,1,10,2020


In [16]:
result=cdf_2020.groupby(["violcode"])['citation'].count().reset_index() #from the year 2020 counting the citations
c_viol=result.loc[result['citation']==max(result['citation'])].reset_index()['violcode'].values ## checking for the most common violation
result['citationpercentage'] = result.citation / sum(result.citation)*100 # to find the percentage of citations
print('Common violation type is :',c_viol)
print('Total citations percentage is:',max(result['citationpercentage']))

Common violation type is : [32]
Total citations percentage is: 60.34460812342532


###c. In 2020, which violation type was responsible for the highest fine revenue (i.e., highest sum of fine amounts)? What percentage of total fine revenue did it comprise for the year?


In [17]:
high_fine=cdf_2020.groupby(["description"])['violfine'].sum().reset_index() #checking the highest fine and associating with the description
result_fine=high_fine.loc[high_fine['violfine']==max(high_fine['violfine'])]
print("Violation Type responsible for highest fine revenue\n",result_fine)
x=result_fine.values[0][1]/sum(high_fine['violfine'])*100 # to find the percentage
print("Percentage of total fine revenue in the year 2020:", x)

Violation Type responsible for highest fine revenue
            description  violfine
10  Fixed Speed Camera  23893520
Percentage of total fine revenue in the year 2020: 48.27915229610332


###d. Show the fines charged against the 10 tags (license plates) with the highest total fines assessed in 2020. For each, show the total number of citations, and mean/standard deviation of the fine amount.

In [39]:
tag_df=cdf_2020.groupby(["tag"])['violfine'].sum().reset_index().sort_values('violfine',ascending=False).head(10) # to find the highest fines adding all the fines for the individual tags.
tag_df


,tag,violfine
365131,A354123,5895
403573,NO TAGST,4777
348182,9DR2114,4740
245282,6EE1679,4630
419499,URX4846,4421
149506,4CXM25,4365
407714,RR5H74,4237
312023,8DW4455,3980
62336,2CC3046,3882
363971,A334199,3812


In [42]:
fine_mean= tag_df.groupby(tag_df['tag'])['violfine'].mean() # finding the mean of the fines
fine_mean

tag
2CC3046     3882.0
4CXM25      4365.0
6EE1679     4630.0
8DW4455     3980.0
9DR2114     4740.0
A334199     3812.0
A354123     5895.0
NO TAGST    4777.0
RR5H74      4237.0
URX4846     4421.0
Name: violfine, dtype: float64

### (e) Due to accuracy concerns, Baltimore City suspended fixed speed camera enforcement for a multi-year period that is covered by the dataset. Using the data, show when the hiatus began and when citations resumed.

In [43]:
cdf_BC = cdf[cdf["state"]=="MD"] #electing the state MD from the complete dataset
cdf_BC.head()

,citation,tag,state,violcode,description,violfine,violdate,location,violdtl,day,month,year
0,91,03W671,MD,27,No Stop/Park Street Cleaning,52,2011-12-09 11:55:00-05:00,<NA>,2011-12-09 11:55:00,9,12,2011
1,166,T294093T,MD,12,No Stopping/Standing Not Tow-Away Zone,32,2011-12-09 12:39:00-05:00,<NA>,2011-12-09 12:39:00,9,12,2011
4,562,9AE2340,MD,18,All Other Parking Meter Violations,32,2011-12-12 10:47:00-05:00,LAFAYETTE MARKET LOT Baltimore MD,2011-12-12 10:47:00,12,12,2011
5,653,SG23690,MD,5,Obstruct/Impeding Movement of Pedestrian,77,2011-12-12 12:17:00-05:00,"(39.28960468, -76.64890053)",2011-12-12 12:17:00,12,12,2011
6,703,7AB5707,MD,27,No Stop/Park Street Cleaning,52,2011-12-29 08:18:00-05:00,"(39.29407531, -76.65950967)",2011-12-29 08:18:00,29,12,2011


In [45]:
speed_camera=cdf_BC.loc[(cdf_BC['description'] == 'Fixed Speed Camera')] # Checking for the description havinf fixed speed Camera 
speed_camera['year'].unique() # From this we can observe that cameras were removed in the period 2013-2017  

array([2017, 2019, 2020, 2012, 2021, 2009, 2010, 2018, 2011])

##Question 3
**Frawley, et al., define an 'interesting' pattern as interesting if it is "novel, useful, and non-trivial to compute."  Explore either of the above datasets to identify an interesting pattern.  Describe the pattern and why it is novel, useful, and non-trivial.**

Note: this question does not require code.  Submit your response as text within the notebook, or as a Word document if you prefer.

### From the above data sets it is observed that, the highest temperature is recorded in 2020 was in the month of July and the coldest was in the month of December with a total of 144 rainy days .We can track a record of data for every year and it can be used to predict the climate changes taking place and also consider what is affecting the climate.

##Question 4

**Martínez-Plumed et al. define a Data Science Trajectories model that builds on the CRISP-DM framework.  For what types of projects would you use CRISP-DM?  In contrast, when would an alternative model like {\sc dst} be more suitable?**

Note: this question does not require code.  Submit your response as text within the notebook, or as a Word document if you prefer.

### Cross Industry Process for Data Mining(CRISP-DM) is a data mining process used for providing the best results in a organization.It has 6 phases which describes life cycle to construct a well organized project.
- Business understanding 
- Data understanding 
- Data preparation 
- Modeling 
- Evaluation
- Deployment
### CRISP-DM is a model which will help in reducing the cost for any data mining projects and also minimum knowledge is required to use this tool.

# Question 5
**Machine Learning, Safety, and Fairness** Review the source materials for one of the examples of unfairness in machine learning discussed in class, and answer the following questions.  

(a) Mehrabi et al. define fairness in machine learning as ``absence of any prejudice or favoritism toward an individual or group based on
their inherent or acquired characteristics''.  Using this definition, how does the application of the machine learning approach result in an unfair outcome?

(b) What practices, if any, mitigated the impact of the algorithm's unfairness?

(c) What lessons can be learned from this example?


### (a).From the paper we can observe that any unfair algorithm may cause a decision completly biased towards any one side which will lead to wrong outcomes.AI Machine learning is extensively used in many industries now a days and if any error or any unfairness will cause many problems.So any developer developing a program should use all the test cases for any product to be fair because Machine learning is used in daily life. From the example provided in UK where the graded providied to the students who are from a poor background is completly wrong because the ML is trained to grade the students incorrectly and it is unfair to grade in such a way based on anyones financial ability.

### (b).By removing the factor of Financial status we can train the product to be completly fair and grade the students based on their performance can  reduce the unfairness.

### (c).By this example we can learn that any product before implementing it should be tested will all the possible outcomes so that it will not lead to any social issue.